In [ ]:
#Configurações iniciais

#Imports do google
import google.generativeai as genai
#from google.colab import userdata


#Imports da biblioteca de filesystem paths
from pathlib import Path


#Imports da comunicação com o Telegram
import requests

#Imports do Servidor FTP
import os
from pyftpdlib.authorizers import DummyAuthorizer
from pyftpdlib.handlers import FTPHandler
from pyftpdlib.servers import FTPServer

#Import da biblioteca responsável pela leitura do arquivo .ini contendo as chaves e tokens
import configparser


config = configparser.ConfigParser()
config.read('config.ini')


#Chaves - Comente essas 3 linhas abaixo em favor das que vem em seguida
api_key = config['DEFAULT']['api_key'] #Chave da API do Google
token_bot = config['DEFAULT']['token_bot'] # Chave do Bot do Telegram
chat_id = config['DEFAULT']['chat_id'] # ID do usuário Telegram que receberá os alertas

#Chaves - Substituia pelas suas informações
#api_key = 'ABC'
#token_bot = '123'
#chat_id = '321'


genai.configure(api_key=api_key)

class MyHandler(FTPHandler):

    # Caminho da imagem
    image_path = ""

    def on_file_received(self, file):
        # Este método é chamado quando um arquivo é recebido com sucesso.
        image_path = file
        print(f"Alerta: Nova imagem recebida - ",image_path)

        # Set up the model
        generation_config = {
        "temperature": 0.9,
        "top_p": 0.95,
        "top_k": 32,
        "max_output_tokens": 1024,
        }

        safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE"
        },
        ]

        model = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest", 
                                      generation_config=generation_config,
                                      safety_settings=safety_settings)
        prompt_parts = [
            "O que há na imagem?", "Há pessoas nela?","Objeto ou pessoa: ",
            genai.upload_file(image_path),
            "Descrição:",
        ]
        
        response = model.generate_content(prompt_parts)

        # Processar a resposta
        objects = response.text.strip().split("\n")
        msg = 'Pessoa detectada na propriedade!'+"\n"+"Descrição: "+response.text
       
        """
            Verifica se o objeto cuja imagem foi processada é uma pessoa
        """
        def is_person(object_description):
            person_keywords = ["pessoa", "pessoas", "homem","homens", "mulher","mulheres", "criança","crianças", "gente", "bandido",
                               "bandidos", "individuo", "humano","humanos", "assaltante", "assaltantes", "ladrão","ladrões","menino",
                               "menina","meninos","meninas","garoto","garotos","garota","garotas"]
            return any(keyword in object_description.lower() for keyword in person_keywords)
        
        """
            Envia uma mensagem com uma imagem para um chat do Telegram.

            Args:
                mensagem: A mensagem a ser enviada.
                caminho_imagem: O caminho para a imagem.
                chat_id: O ID do chat para o qual a mensagem será enviada.
                token_bot: O token do bot do Telegram.
        """
        def enviar_mensagem_telegram_com_imagem(mensagem, caminho_imagem, chat_id, token_bot):
            url = f"https://api.telegram.org/bot{token_bot}/sendPhoto"
            files = {"photo": open(caminho_imagem, "rb")}
            data = {"chat_id": chat_id, "caption": mensagem}
            requests.post(url, files=files, data=data)
            print("Mensagem com imagem enviada com sucesso!")

        for obj in objects:
            if is_person(obj):
                enviar_mensagem_telegram_com_imagem(msg, image_path, chat_id, token_bot)
                break  # Sair do loop após enviar a mensagem
            else:
                print("Nenhuma pessoa detectada!")
        print(response.text)        
      

def main():
    authorizer = DummyAuthorizer()
    authorizer.add_user("user", "12345", ".", perm="elradfmwMT") # substitua "user" e "12345" pelo seu nome de usuário e senha

    handler = MyHandler
    handler.authorizer = authorizer

    server = FTPServer(("localhost", 2121), handler) # substitua "localhost" e "2121" pelo seu endereço IP e porta
    server.serve_forever()

if __name__ == "__main__":
    main()
